In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ita-dpo-2/adapter_model.safetensors
/kaggle/input/ita-dpo-2/trainer_state.json
/kaggle/input/ita-dpo-2/training_args.bin
/kaggle/input/ita-dpo-2/adapter_config.json
/kaggle/input/ita-dpo-2/README.md
/kaggle/input/ita-dpo-2/tokenizer.json
/kaggle/input/ita-dpo-2/tokenizer_config.json
/kaggle/input/ita-dpo-2/scaler.pt
/kaggle/input/ita-dpo-2/scheduler.pt
/kaggle/input/ita-dpo-2/special_tokens_map.json
/kaggle/input/ita-dpo-2/optimizer.pt
/kaggle/input/ita-dpo-2/rng_state.pth
/kaggle/input/ita-dpo-2/tokenizer.model
/kaggle/input/ita-dpo-3/adapter_model.safetensors
/kaggle/input/ita-dpo-3/trainer_state.json
/kaggle/input/ita-dpo-3/training_args.bin
/kaggle/input/ita-dpo-3/adapter_config.json
/kaggle/input/ita-dpo-3/README.md
/kaggle/input/ita-dpo-3/tokenizer.json
/kaggle/input/ita-dpo-3/tokenizer_config.json
/kaggle/input/ita-dpo-3/scaler.pt
/kaggle/input/ita-dpo-3/scheduler.pt
/kaggle/input/ita-dpo-3/special_tokens_map.json
/kaggle/input/ita-dpo-3/optimizer.pt
/kaggle/input/

In [3]:
!pip install datasets evaluate transformers accelerate peft trl 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 7.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 24.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.1 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
  

In [4]:
# Cell 1: Imports and Setup
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import DPOTrainer, DPOConfig
from peft import LoraConfig
import torch


2025-06-10 11:43:20.013881: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749555800.328229      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749555800.417817      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
# Cell 2: Load tokenizer and model

model1_path = "/kaggle/input/tinyllama-empathetic-sft-v7"
tokenizer = AutoTokenizer.from_pretrained(model1_path, use_fast=True)

sft_model = AutoModelForCausalLM.from_pretrained(
    model1_path,
    device_map=None,          # disables automatic device placement
    torch_dtype=torch.float32 # use float32 for CPU compatibility
).to("cpu") 

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [17]:
from peft import PeftModel

base_model = PeftModel.from_pretrained(
    sft_model,
    "/kaggle/input/ita-dpo-4",
    torch_dtype=torch.float32
).to("cpu")
base_model.train()
# List all modules with trainable params
for name, param in base_model.named_parameters():
    if param.requires_grad:
        print(f"Trainable: {name}")

print(f"Trainable params: {sum(p.numel() for p in base_model.parameters() if p.requires_grad)}")

Trainable params: 0


/usr/local/lib/python3.11/dist-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight', 'base_model.model.model.layers.0.mlp.gate_proj.lora_A.default.weight', 'base_model.model.model.layers.0.mlp.gate_proj.lora_B.default.weight', 'base_model.model.model.layers.0.mlp.up_proj.lora_A.default.weight', 'base_model.model.model.layers.0.mlp.up_proj.lora_B.default.weight', 'base_model.model.model.layers.0.mlp.down_proj.lora_A.default.weight', 'base_model.model.model.layers.0.mlp.down_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.1.self_attn.k_proj.lora_B.default.weight',

In [18]:
from peft import PeftModel, PeftConfig, LoraConfig, get_peft_model, TaskType

base_model.load_adapter("/kaggle/input/ita-dpo-4", adapter_name="def4")
base_model.set_adapter("def4")

# Set the model to eval or train as needed
base_model.eval()

base_model.print_trainable_parameters()

trainable params: 2,252,800 || all params: 1,112,799,232 || trainable%: 0.2024


In [19]:
from peft import PeftModel

fine_tuned_model = PeftModel.from_pretrained(
    sft_model,
    "/kaggle/input/ita-dpo-5",
    torch_dtype=torch.float32
).to("cpu")
fine_tuned_model.train()
# List all modules with trainable params
for name, param in fine_tuned_model.named_parameters():
    if param.requires_grad:
        print(f"Trainable: {name}")

print(f"Trainable params: {sum(p.numel() for p in fine_tuned_model.parameters() if p.requires_grad)}")

Trainable params: 0


In [20]:
from peft import PeftModel, PeftConfig, LoraConfig, get_peft_model, TaskType

fine_tuned_model.load_adapter("/kaggle/input/ita-dpo-5", adapter_name="def5")
fine_tuned_model.set_adapter("def5")

# Set the model to eval or train as needed
fine_tuned_model.eval()

fine_tuned_model.print_trainable_parameters()

trainable params: 1,531,904 || all params: 1,113,204,736 || trainable%: 0.1376


In [21]:
pairs = [
    (
        "<|system|>\nRespond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nI feel like no one really understands what I’m going through.\n"
        "<|assistant|>\n",
        "I'm really sorry you're feeling that way. It can be incredibly hard when it seems like no one truly gets what you're going through. You're not alone, and your feelings are valid. I'm here to listen and support you."
    ),
    (
        "<|system|>\nRespond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nMy best friend hasn’t talked to me in days, and it’s making me anxious.\n"
        "<|assistant|>\n",
        "That sounds really tough. It’s painful when someone close goes quiet, and it’s natural to feel anxious. Remember, their silence doesn’t define your worth. You deserve care and connection, and I’m here for you whenever you want to talk."
    ),
    (
        "<|system|>\nRespond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nI’m overwhelmed with school and feel like I’m falling behind.\n"
        "<|assistant|>\n",
        "I hear you, it’s so overwhelming when school piles up and you feel like you’re slipping behind. It’s okay to feel this way, and it doesn’t mean you’re failing. Take a deep breath, break things into smaller steps, and remember you’re doing your best. You’ve got this."
    ),
    (
        "<|system|>\nRespond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nI tried really hard on my project, but it didn’t turn out the way I hoped.\n"
        "<|assistant|>\n",
        "That’s really disappointing, and I’m sorry your project didn’t go as you hoped. Your effort matters so much, and setbacks don’t erase all the hard work you put in. Every experience is a step forward, even if it doesn’t feel that way now. Keep believing in yourself."
    ),
    (
        "<|system|>\nRespond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nI got some bad news about a family member today.\n"
        "<|assistant|>\n",
        "I’m so sorry to hear that. Receiving tough news about someone you love can feel overwhelming and heavy. It’s okay to feel whatever comes up right now, grief, fear, confusion. You’re not alone, and I’m here to support you through this."
    ),
    (
        "<|system|>\nRespond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nI feel really lonely these days.\n"
        "<|assistant|>\n",
        "Feeling lonely can be so painful, and I’m sorry you’re going through this. Remember, your feelings matter, and it’s okay to reach out, even just a little. You’re not alone, and there are people who care deeply about you. I’m here to listen whenever you need."
    ),
    (
        "<|system|>\nRespond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nEveryone expects me to be strong, but I’m really struggling inside.\n"
        "<|assistant|>\n",
        "That’s incredibly hard, to carry the weight of others’ expectations while struggling inside. It’s okay to admit you’re not okay. Your strength includes being honest about your feelings, and you don’t have to face this alone. You deserve support and kindness."
    ),
    (
        "<|system|>\nRespond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nMy partner and I had a big argument and now I feel emotionally drained.\n"
        "<|assistant|>\n",
        "I’m really sorry you’re feeling this way. Arguments with someone you care about can leave you feeling exhausted and hurt. It’s okay to take time to breathe and care for yourself. Your feelings are valid, and healing takes time. You’re not alone."
    ),
    (
        "<|system|>\nRespond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nI’m scared about what the future holds. Everything feels uncertain.\n"
        "<|assistant|>\n",
        "Feeling scared about the future is completely natural when everything feels uncertain. It’s okay to not have all the answers right now. Take things one step at a time, and be gentle with yourself. You’re stronger than you realize, and I’m here with you through this."
    ),
    (
        "<|system|>\nRespond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nI just feel like giving up lately.\n"
        "<|assistant|>\n",
        "I’m really sorry you’re feeling this way. It’s okay to feel overwhelmed and unsure, but please remember you’re not alone and your feelings matter. Reaching out for support can make a difference, you deserve kindness and hope, even when it’s hard to see."
    ),
]


In [22]:
def collect_model_responses_for_human_eval(pairs, base_model, fine_tuned_model, tokenizer, device):
    results = []

    # Set models to evaluation mode
    base_model.eval()
    fine_tuned_model.eval()


    # Activate adapter if available
    if hasattr(base_model, "set_adapter"):
        base_model.set_adapter("def4")
        
    # Activate adapter if available
    if hasattr(fine_tuned_model, "set_adapter"):
        fine_tuned_model.set_adapter("def5")

    for i, (prompt, expected_response) in enumerate(pairs):
        # Tokenize without truncating, no padding (to preserve prompt shape)
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

        with torch.no_grad():
            # Generate from base model
            base_output = base_model.generate(
                input_ids=input_ids,
                max_new_tokens=150,
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )
            base_response = tokenizer.decode(base_output[0], skip_special_tokens=True)

            # Generate from fine-tuned model (DPO)
            fine_output = fine_tuned_model.generate(
                input_ids=input_ids,
                max_new_tokens=150,
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )
            fine_response = tokenizer.decode(fine_output[0], skip_special_tokens=True)

        results.append({
            "index": i,
            "prompt": prompt,
            "expected_response": expected_response,
            "base_response": base_response,
            "fine_tuned_response": fine_response
        })

    return results


In [23]:
device = "cpu" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cpu


In [24]:
human_eval_data = collect_model_responses_for_human_eval(
    pairs=pairs,
    base_model=base_model,
    fine_tuned_model=fine_tuned_model,
    tokenizer=tokenizer,
    device=device
)

In [25]:
# Example: Print nicely for human review
for item in human_eval_data:
    print(f"\n--- Example {item['index'] + 1} ---")
    print(f"Prompt:\n{item['prompt']}")
    print(f"\nExpected Response:\n{item['expected_response']}")
    print(f"\nDPO4 Response:\n{item['base_response']}")
    print(f"\nDPO5 Response:\n{item['fine_tuned_response']}")
    print("\n" + "-" * 60)



--- Example 1 ---
Prompt:
<|system|>
Respond to this empthatically,Respond without using emoticons.
<|user|>
I feel like no one really understands what I’m going through.
<|assistant|>


Expected Response:
I'm really sorry you're feeling that way. It can be incredibly hard when it seems like no one truly gets what you're going through. You're not alone, and your feelings are valid. I'm here to listen and support you.

DPO4 Response:
<|system|>
Respond to this empthatically,Respond without using emoticons.
<|user|>
I feel like no one really understands what I’m going through.
<|assistant|>
I understand that feeling. It’s common to feel like no one really understands what you’re going through, but that doesn't have to be the case. I’m here to help you. Empathy is the ability to put yourself in someone else's shoes and understand their perspective. It's about being attentive and paying attention to what someone else is saying or feeling. When you empathize with someone who is going throu

In [26]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.tokenize import word_tokenize
import nltk
nltk.download("punkt")

# Set device
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")

# BLEU smoother
smoothie = SmoothingFunction().method4

def generate_response(model, prompt):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    output = model.generate(
        input_ids,
        max_length=150,
        temperature=0.7,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True,  # Usually helps diversity
        top_p=0.9        # Optional nucleus sampling
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

def compute_bleu(reference_text, candidate_text):
    reference_tokens = word_tokenize(reference_text)
    candidate_tokens = word_tokenize(candidate_text)
    return sentence_bleu([reference_tokens], candidate_tokens, smoothing_function=smoothie)

def evaluate_model_bleu(pairs):
    results = []
    base_total = 0.0
    fine_tuned_total = 0.0

    for i, (prompt, expected_response) in enumerate(pairs):
        base_response = generate_response(base_model, prompt)
        fine_tuned_response = generate_response(fine_tuned_model, prompt)

        base_bleu = compute_bleu(expected_response, base_response)
        fine_tuned_bleu = compute_bleu(expected_response, fine_tuned_response)

        base_total += base_bleu
        fine_tuned_total += fine_tuned_bleu

        results.append({
            "index": i,
            "prompt": prompt,
            "expected_response": expected_response,
            "base_response": base_response,
            "fine_tuned_response": fine_tuned_response,
            "base_bleu": base_bleu,
            "fine_tuned_bleu": fine_tuned_bleu
        })

    avg_base_bleu = base_total / len(pairs)
    avg_fine_tuned_bleu = fine_tuned_total / len(pairs)

    return results, avg_base_bleu, avg_fine_tuned_bleu


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [27]:
results, avg_base, avg_fine_tuned = evaluate_model_bleu(pairs)

SFT7 Avg BLEU: 0.039340276866260585
DPO1 Avg BLEU: 0.032849301728591046


In [28]:
print("DPO4 Avg BLEU:", avg_base)
print("DPO5 Avg BLEU:", avg_fine_tuned)

DPO4 Avg BLEU: 0.039340276866260585
DPO5 Avg BLEU: 0.032849301728591046
